In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import RMSprop ,SGD
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau


# UDF
from eXplainableAI.transparent_model.RNN import ConditionalRETAIN
from eXplainableAI.transparent_model.RNN.interpretation import Interpreter

### Load data

Sorry not to provide data set. This is because of ethic issue. (IRB not permit data sharing to third party )

In [ ]:
DATA_PATH = ''

# Train data
X_train_var = np.load(DATA_PATH + '/cache/numpy/X_train_var.npy')
X_train_inv = np.load(DATA_PATH + '/cache/numpy/X_train_fix.npy')
Y_train = np.load(DATA_PATH + '/cache/numpy/Y_train.npy')

# test
X_test_var = np.load(cwd + '/cache/numpy/X_test_var.npy')
X_test_inv = np.load(cwd +'/cache/numpy/X_test_fix.npy')
Y_test = np.load(cwd + '/cache/numpy/Y_test.npy')
print(X_train_var.shape, X_train_inv.shape)

### Build model

In [ ]:
config = {'n_features':vars_,
          'n_auxs':9,
          'steps':16,
          'hidden_units': 20
          }

conditional_retain = ConditionalRETAIN(config)
retain = conditional_retain.build_model()
retain.compile(optimizer=RMSprop(lr=0.0005), loss='mse', metrics=['mse'])
retain.summary()

In [ ]:
model_dir = './model/'
check_point = ModelCheckpoint(filepath=model_dir+'{epoch:03d}.hdf5',
                     monitor='val_loss', 
                     verbose=1, 
                     save_best_only=True, 
                     save_weights_only=True,
                     mode='min' 
                     )

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.000001, mode='min')  

history = retain.fit(x=[X_train_var, X_train_inv], 
                     y=Y_train,   
                     batch_size=16*5,
                     epochs=500,
                     shuffle=False,
                     callbacks=[check_point, reduce_lr],
                     validation_split=0.3)

### Interpetation on overall data set 
(This is not instance level interpretation like previous RETAIN)

In [ ]:
y_predict = model.predict(x=[X_test_var, X_test_inv])

###### Interpetation: alpha (Time level attention weight)

In [ ]:
interpreter = Interpreter(model, X_test_var, X_test_inv, Y_test)
alpha = np.array(interpreter.get_model_weight('alpha'))

alpha_mean = alpha.reshape(-1, 16, 16, 1)[:, -1, :, :].mean(axis=0)
print('Alpha shape: ', alpha_mean.shape)

fig, axes = plt.subplots(1, 1, figsize=(14, 6))
sns.heatmap(alpha_mean.T, annot=True, fmt='.4f', xticklabels=list(range(1, 17)))

###### Interpetation: Beta (Variable level attention weight)

In [ ]:
beta = np.array(interpreter.get_model_weight('beta'))
beta = beta.reshape(-1, 16, 16, len(vars_))
beta_mean = beta[:, 15, :, :].mean(axis=0)

fig, axes = plt.subplots(1, 1, figsize=(13, 6))
sns.heatmap(beta_mean.T, 
            cmap='coolwarm', 
            annot=True, 
            vmin=-1, 
            vmax=1, 
            xticklabels=list(range(1, 17)),
            yticklabels=vars_
           )
plt.xticks(rotation=45)

###### Fusion layer including static varaibles weight

In [ ]:
W, bias = interpreter.get_model_weight('weight')
W.reshape(-1, W.shape[0])[0][2:4] = W.reshape(-1, W.shape[0])[0][2:4] * 1000  # # calories, steps * 1K당으로 변경


fig, axes = plt.subplots(1, 1, figsize=(18, 6))
sns.heatmap(W.reshape(-1, W.shape[0]), cmap='coolwarm', fmt='.3f', annot=True, 
            xticklabels=vars_+inv_,
            vmin=-1, vmax=1)

fig.savefig('./results/denselayer.png')

###### Overall Coef contribution

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(14, 7))

W_time = W[:len(vars_)]
contr_coef = W_time.ravel() * (alpha_mean * beta_mean)

sns.heatmap(contr_coef.T, cmap='coolwarm', annot=True, fmt='.3f',
            yticklabels=vars_,
            xticklabels=list(range(1, 17)),
            vmin=-0.06, 
            vmax=0.06)